# World countries CO2 emmission rates

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data_path = "./kc_house_data.csv"
data = pd.read_csv(data_path)
data = data.dropna(axis=0)
print(data.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


## Set y and X values

In [3]:
y = data["price"].astype(np.float32)
data_prediction_features = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built"]
X = data[data_prediction_features].astype(np.float32)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Create Model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression task, so no activation function
])

2023-08-13 13:58:44.311192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
loss_fn = tf.keras.losses.MeanSquaredError()
# ues adam optimizer function for dynamically updating gradient descent.
optimizerFunction = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.995,
    epsilon=5e-06,
    amsgrad=True,
    name='Adam',
)
model.compile(optimizer=optimizerFunction,
              loss=loss_fn)

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=15)
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[callback])


Epoch 1/100
487/487 [==============================] - 1s 2ms/step - loss: 186745700352.0000 - val_loss: 52004323328.0000
Epoch 2/100
487/487 [==============================] - 1s 1ms/step - loss: 52189220864.0000 - val_loss: 40235589632.0000
Epoch 3/100
487/487 [==============================] - 1s 1ms/step - loss: 47315804160.0000 - val_loss: 39804284928.0000
Epoch 4/100
487/487 [==============================] - 1s 1ms/step - loss: 46262116352.0000 - val_loss: 37356437504.0000
Epoch 5/100
487/487 [==============================] - 1s 2ms/step - loss: 45697384448.0000 - val_loss: 36765175808.0000
Epoch 6/100
487/487 [==============================] - 1s 1ms/step - loss: 44851281920.0000 - val_loss: 36863528960.0000
Epoch 7/100
487/487 [==============================] - 1s 1ms/step - loss: 44972736512.0000 - val_loss: 36534181888.0000
Epoch 8/100
487/487 [==============================] - 1s 1ms/step - loss: 44375048192.0000 - val_loss: 36183543808.0000
Epoch 9/100
487/487 [==========

In [9]:
loss = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)

136/136 [==============================] - 0s 945us/step - loss: 42195746816.0000
Test loss: 42195746816.0


In [15]:
user_input = [3, 1.75, 3000, 30000, 2, 0, 0, 10, 7, 3000, 0, 2015]

# Normalize the user input using the same scaler
user_input_scaled = scaler.transform(np.array(user_input).reshape(1, -1))

# Predict the price using the trained model
predicted_price = model.predict(user_input_scaled)[0][0]

print(f"Estimated price: ${predicted_price:.2f}")

1/1 [==============================] - 0s 16ms/step
Estimated price: $1062336.88


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
